# This script calculates cAMP ratios acquired using the PaQa plasmid and pre-analysed using stock BacStalk

* Change all directories in this script according to your system
* Record images of your cells containing the PaQa plasmid
* Segment cells with BacStalk (v1.8, https://drescherlab.org/data/bacstalk/docs/index.html)
  * Recommended to name the Channels "PaQa" for YFP fluorescence and "RFP" for mKate2 fluorescence (in case of PaQa plasmid)
  * You can use the ImageJ macro "SplitImage_MJK.ijm" in the static_localization folder on GitHub (https://github.com/PersatLab/antagonists) to split multichannel images to use for BacStalk
  * Export one csv file per sample containing at least: MeanCellIntensity_PaQa, MeanCellIntensity_RFP ("..._PaQa" and "..._RFP" are the names of the channels you enter in BacStalk)
  * Save BacStalk file if you want to go back and change some analysis parameters 
  * !!! Important !!! Format of the names of the csv files must be:
    * date_strain-number_Data_strain-name_PaQa_condition_replicate
    * no underscores within the strain name or other items
    * conditions must include liq or sol, because this script looks for "liq" or "sol" in the file name
    * same for "rep"
    * example: 20220421_1591_Data_fliC-chpALOF-cpdA-_PaQa_3hsol_rep2.csv
* Run this script, modify directories according to your system, and enter reference strain and strains to plot in function "plottingData" (last cell)

In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from bokeh.models import Range1d
from scipy import stats
import pandas as pd
import math

from bokeh.layouts import row

bokeh.io.output_notebook()

Loading BokehJS ...

In [64]:
# defines how cAMP ratios are calculated and plotted
# normally, no need to modify anything

def plottingAllData(df, means, labels, indexes, param, colors):
    df.head()
    p = bokeh.plotting.figure(
        width=600, 
        height=600, 
        y_range=labels, 
        x_axis_type='linear',
        x_axis_label = 'Fluorescence intensity',
        title="cAMP "+param+" on solid vs liquid environement"
    )
    for i, index in enumerate(indexes):
        p.circle(
            source=df.loc[index, :],
            x=param, 
            y=jitter('Labels', width=0.3, range=p.y_range),
            color = colors[i],
            alpha=0.3,
            #legend = labelsAll[i]
        )
    p.circle(
        source = means,
        x = param,
        y = 'Labels',
        size = 10,
        line_color = 'black',
        fill_color = 'white',
        legend = "medians"
    )
    p.add_tools(HoverTool(
            tooltips=[
                ('Strain', '@{Strain}'),
                ('Growth condition', '@{Growth}'),
                (param, '@{'+param+'}'),
                ('Biological replicate', '@{Bio_Rep}')
            ],
    ))
    return p

def plottingData(df, means, wt, order, strain_to_remove, growth, param, colors, scale, GraphTitle):
    df2 = df.copy()
    means2 = means.copy()
    
    if isinstance(growth, list):
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.copy()
            means3 = means2.copy()
        val = 0.5
        labels = [None]*(2+len(order)*2)
        indexes = [None]*(2+len(order)*2)
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        val = val + 2
        n=2
        for o, ordering in enumerate(order):
            for a, g in enumerate(growth):
                df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(g)), 'Labels'] = val
                means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(g)), 'Labels'] = val
                labels[n] = ordering + ' ' +g
                indexes[n] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(g))
                val = val + 1
                n=n+1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            #width=600, 
            width=1000, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth[0]+" vs. "+growth[1]+" in "+scale+" scale"
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    else:
        print('Not a list')
        if growth == 'Liquid':
            ungrowth = 'Solid'
        else:
            ungrowth = 'Liquid'
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.drop(df2[((df2['Strain'] == strain) & (df2['Growth'] == ungrowth))].index)
            means2 = means2.drop(means2[((means2['Strain'] == strain) & (means2['Growth'] == ungrowth))].index)
        means3=means2
        val = 0.5
        labels = [None]*(2+len(order))
        indexes = [None]*(2+len(order))
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        WT_mean=np.array(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param])
        Norm_mean=WT_mean/np.mean(WT_mean)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param]/np.mean(WT_mean)
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        print(wt+' Liquid mean='+str(WT_mean))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        Strain_mean=np.array(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), param])
        Norm_mean=Strain_mean/WT_mean
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), param]/np.mean(WT_mean)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        print(wt+' Solid mean='+str(Strain_mean))
        print(wt+' Solid fold increase='+str(Strain_mean/np.mean(WT_mean)))
        val = val + 2
        for o, ordering in enumerate(order):
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), 'Labels'] = val
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), 'Labels'] = val
            Strain_mean=np.array(means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param])
            Norm_mean=Strain_mean/np.mean(WT_mean)
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param]=Norm_mean
            print(ordering+' mean='+str(Strain_mean))
            print(ordering+' fold increase='+str(Strain_mean/np.mean(WT_mean)))
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param] = df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param]/np.mean(WT_mean)
            labels[o+2] = ordering+' '+growth
            indexes[o+2] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(growth))
            val = val + 1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fold increase',
            title=GraphTitle+" "+param+" on "+growth
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            #legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    return p, means3

def plottingDataUnique(df, means, wt, order, strain_to_remove, growth, param, colors, scale, GraphTitle):
    df2 = df.copy()
    means2 = means.copy()
    
    if isinstance(growth, list):
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.copy()
            means3 = means2.copy()
        val = 0.5
        labels = [None]*(2+len(order)*2)
        indexes = [None]*(2+len(order)*2)
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        df3.loc[((df2['Strain'] == (wt)) & df3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Solid')), 'Labels'] = (val+1)
        labels[1] = wt+' '+'Solid'
        indexes[1] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Solid'))
        val = val + 2
        n=2
        for o, ordering in enumerate(order):
            for a, g in enumerate(growth):
                df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(g)), 'Labels'] = val
                means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(g)), 'Labels'] = val
                labels[n] = ordering + ' ' +g
                indexes[n] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(g))
                val = val + 1
                n=n+1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth[0]+" vs. "+growth[1]+" in "+scale+" scale"
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"
        for i, index in enumerate(indexes):
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_lable = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    else:
        if growth == 'Liquid':
            ungrowth = 'Solid'
        else:
            ungrowth = 'Liquid'
        for strain in strain_to_remove:
            df2 = df2.drop(df2[(df2['Strain'] == strain)].index)
            means2 = means2.drop(means2[(means2['Strain'] == strain)].index)
        for strain in order:
            df3 = df2.drop(df2[((df2['Strain'] == strain) & (df2['Growth'] == ungrowth))].index)
            means3 = means2.drop(means2[((means2['Strain'] == strain) & (means2['Growth'] == ungrowth))].index)
        val = 0.5
        labels = [None]*(1+len(order))
        indexes = [None]*(1+len(order))
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), 'Labels'] = val
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), 'Labels'] = val
        WT_mean=list(means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param])
        Norm_mean=WT_mean/WT_mean
        means3.loc[((means3['Strain'] == (wt)) & means3['Growth'].str.match('Liquid')), param]=Norm_mean
        df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param] = df3.loc[((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid')), param]/WT_mean
        labels[0] = wt+' '+'Liquid'
        indexes[0] = ((df3['Strain'] == (wt)) & df3['Growth'].str.match('Liquid'))
        val = val + 1
        for o, ordering in enumerate(order):
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), 'Labels'] = val
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), 'Labels'] = val
            Strain_mean=float(means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param])
            Norm_mean=Strain_mean/WT_mean
            means3.loc[((means3['Strain'] == (ordering)) & means3['Growth'].str.match(growth)), param]=Norm_mean
            df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param] = df3.loc[((df2['Strain'] == (ordering)) & df3['Growth'].str.match(growth)), param]/WT_mean
            labels[o+1] = ordering+' '+growth
            indexes[o+1] = ((df3['Strain'] == (ordering)) & df3['Growth'].str.match(growth))
            val = val + 1
        lookupdf2=df3.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
        p = bokeh.plotting.figure(
            width=600, 
            height=600, 
            x_range=labels, 
            y_axis_type=scale,
            y_axis_label = 'Fluorescence intensity',
            title=GraphTitle+" "+param+" on "+growth
        )
        p.xaxis.major_label_orientation = math.pi/4
        p.xaxis.major_label_text_font_size = "9pt"       
        for i, index in enumerate(indexes):
            print(df3.loc[index, param].mean())
            p.circle(
                source=df3.loc[index, :],
                x=jitter('Labels', width=0.3, range=p.y_range),
                y=param, 
                color = colors[i],
                alpha=0.3,
                #legend = labelsAll[i]
            )
        print(means3)
        p.circle(
            source = means3,
            x = 'Labels',
            y = param,
            size = 10,
            line_color = 'black',
            fill_color = 'white',
            legend_label = "medians"
        )
        p.add_tools(HoverTool(
                tooltips=[
                    ('Strain', '@{Strain}'),
                    ('Growth condition', '@{Growth}'),
                    (param, '@{'+param+'}'),
                    ('Biological replicate', '@{Bio_Rep}')
                ],
        ))
    return p

def getDFStats(df, param):
    Strains=list(df.Strain.unique())
    Growth=list(df.Growth.unique())
    Reps=list(df.Bio_Rep.unique())
    inds = [None]*(len(Strains)*len(Growth))
    labels = [None]*(len(Strains)*len(Growth))
    labelsAll = [None]*(len(Strains)*len(Growth)*len(Reps))
    indexes = [None]*(len(Strains)*len(Growth)*len(Reps))
    n = 0
    m = 0
    for strain in Strains:
        for growth in Growth:
            inds[n] = (df['Strain'] == strain) & (df['Growth'] == growth)
            labels[n] = strain+' '+growth
            n= n + 1
            for rep in Reps:
                indexes[m] = (df['Strain'] == strain) & (df['Growth'] == growth) & (df['Bio_Rep'] == rep)
                labelsAll[m] = strain+' '+growth
                m = m + 1
    df['Labels'] = 0
    #df.insert(2, "Labels", 0, True)
    vertical_position = [None]*len(labels)
    val=0.5        
    for s, strain in enumerate(labels):
        vertical_position[s] = val
        df.loc[inds[s], 'Labels'] = val
        val = val+1        
    means = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].agg(['median', 'count']).rename(columns={'median': param, 'count': 'N'}).reset_index()
    #means = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].median().to_frame().reset_index()
    stds = df.groupby(['Strain', 'Growth', 'Labels', 'Bio_Rep'])[param].std().to_frame().reset_index()
    return means, stds, indexes, labels

In [3]:
data_path = 'H:/directory/BacStalkAnalysis/' # this is where you save your BacStalk csv files, make that folder before running!
working_path = "H:/directory/PythonPaQaAnalysis/" # this is where this script saves enhanced csv files, make that folder before running!

os.chdir(working_path)
new_dir = "date\\" # enter date or other name for your project, this will be inside the PythonPaQaAnalysis folder
working_data_path = working_path + new_dir
if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    
os.chdir(data_path)
extension = 'csv'
list_csv = []
for root, dirs, files in os.walk(data_path, topdown=False):
    for name in files:
        if extension in name:
            list_csv.append(os.path.join(root, name))
#print(list_csv)


for file in list_csv:
    df_temp = pd.read_csv(file, sep=',', na_values='*')
    split_name=file.rsplit('Data_',2)[1].rsplit('_',100) # splits the name after 'Data_', then takes second part and splits after each '_'
    #print(file)
    #print(split_name)
    file_name = file.rsplit("BacStalkAnalysis/",2)[1] # original: file.rsplit("\\",2)[2] -> didn't work for my naming
    #print("Filename: ",file_name)
    df_temp['Strain'] = split_name[0]
    if 'sol' in file:
        df_temp['Growth']='Solid'
    else:
        df_temp['Growth']='Liquid'
    df_temp['Bio_Rep']=int(split_name[len(split_name)-1].rsplit('rep', 2)[1].rsplit('.',2)[0])
    df_temp.to_csv(working_data_path + 'enhanced_' + file_name, index = None, header=True)
    list_enhanced_csv = []

    

for root, dirs, files in os.walk(working_data_path, topdown=False):
    for name in files:
        if extension in name:
            list_enhanced_csv.append(os.path.join(root, name))

combined_csv = pd.concat([pd.read_csv(f) for f in list_enhanced_csv ], sort=False)
os.chdir(working_path)
combined_csv.to_csv( "SummaryPaQa.csv", index=False, encoding='utf-8-sig')

In [4]:
# if you named your channels differently in BacStalk, rename them here below!

df_long = pd.read_csv('SummaryPaQa.csv', sep=',', na_values='*')
df_long['PaQa_ratio']=df_long['MeanCellIntensity_PaQa']/df_long['MeanCellIntensity_RFP'] # rename if different channel names in BacStalk

df_long.head()

,Frame,CellID,Area,CellLength,CellSinuosity,CellWidth,MaxCellIntensity_PaQa,MeanCellIntensity_PaQa,MedianCellIntensity_PaQa,MinCellIntensity_PaQa,MaxCellIntensity_RFP,MeanCellIntensity_RFP,MedianCellIntensity_RFP,MinCellIntensity_RFP,Strain,Growth,Bio_Rep,PaQa_ratio
0,1,2,2.657525,3.711471,1.074485,1.072007,275,160.441971,165.0,41,533,307.972973,315.0,67,fliC-,Liquid,1,0.520961
1,1,4,2.970175,3.942696,1.025728,1.009071,412,258.887624,266.0,91,570,383.910384,401.0,110,fliC-,Liquid,1,0.674344
2,1,8,2.416700,3.277289,1.050185,1.105000,221,129.302448,133.0,38,596,377.045455,386.0,98,fliC-,Liquid,1,0.342936
3,1,9,2.615275,3.537289,1.060552,1.105000,610,402.760905,421.0,153,607,368.552504,380.0,87,fliC-,Liquid,1,1.092818
4,1,10,2.632175,3.541481,1.055414,1.118302,428,288.343499,299.0,92,522,314.330658,336.0,60,fliC-,Liquid,1,0.917325


In [5]:
param='PaQa_ratio' # this is the name of the main parameter that is going to be plotted, can be changed to different parameters in principle
    
[means, stds, indexes, labels] = getDFStats(df_long, param)

means.head(100)

,Strain,Growth,Labels,Bio_Rep,PaQa_ratio,N
0,fliC-,Liquid,0.5,1,0.854507,509
1,fliC-,Liquid,0.5,2,0.872512,286
2,fliC-,Liquid,0.5,3,0.942516,561
3,fliC-,Solid,1.5,1,2.165352,357
4,fliC-,Solid,1.5,2,2.834234,196
5,fliC-,Solid,1.5,3,2.567194,119
6,fliC-chpA-,Liquid,6.5,1,0.249046,288
7,fliC-chpA-,Liquid,6.5,2,0.251710,205
8,fliC-chpA-,Solid,7.5,1,0.235964,91
9,fliC-chpA-,Solid,7.5,2,0.236208,334


In [71]:
# This adds the number of segmented cells to the dataframe

N_sum = np.zeros(len(means))

i = 0
for strain in np.unique(means["Strain"]):    
    for condition in np.unique(means["Growth"][means["Strain"]==strain]):
        N_list = means["N"][means["Strain"]==strain][means["Growth"]==condition]
        summation = np.sum(N_list)
        
        N_sum[i] = summation
        i = i + len(N_list)
    
means["N_sum"] = N_sum.astype('int')

means.head(100)

,Strain,Growth,Labels,Bio_Rep,PaQa_ratio,N,Nsum,N_sum
0,fliC-,Liquid,0.5,1,0.854507,509,1356,1356
1,fliC-,Liquid,0.5,2,0.872512,286,1356,0
2,fliC-,Liquid,0.5,3,0.942516,561,1356,0
3,fliC-,Solid,1.5,1,2.165352,357,1356,672
4,fliC-,Solid,1.5,2,2.834234,196,1356,0
5,fliC-,Solid,1.5,3,2.567194,119,1356,0
6,fliC-chpA-,Liquid,6.5,1,0.249046,288,1356,493
7,fliC-chpA-,Liquid,6.5,2,0.251710,205,1356,0
8,fliC-chpA-,Solid,7.5,1,0.235964,91,1356,425
9,fliC-chpA-,Solid,7.5,2,0.236208,334,1356,0


In [75]:
# This finally plots the datapoints (as individual points, very slow when too many)

colors = bokeh.palettes.d3['Category20'][20]*3
growth=['Liquid','Solid']

# format for following line: (df_long, means, 'reference strain', ['list of strains to plot'], , ['list of strains to ignore'], growth, param, colors, 'linear', '')
p1, me1 = plottingData(df_long, means, 'fliC-', ['fliC-chpA-','fliC-chpA-cpdA-','fliC-chpALOF','fliC-chpALOF-cpdA-','fliC-cyaB-','fliC-cpdA-'], [], growth, param, colors, 'linear', '')

p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
p1.xaxis.minor_tick_line_color = None
p1.yaxis.minor_tick_line_color = None
p1.y_range=Range1d(0,20) # set y-axis
p1.output_backend = 'svg'
bokeh.io.show(p1)